In [18]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [19]:
reload(pi)
reload(pi.strategies)
reload(pi.strategies.buy_x_week_low)
reload(pi.strategies.buy_close_trail)
reload(pi.summarize.trades)
reload(pi.indicators)

<module 'polars_indicators.indicators' from 'c:\\users\\avery\\documents\\github\\polar-indicators\\src\\polars_indicators\\indicators\\__init__.py'>

In [20]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [21]:
saved_df = df.clone()

In [22]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))

In [23]:
lower_limit = 3
upper_limit = 250
close_within_limits = f"close_over{lower_limit}_under{upper_limit}"
df = df.with_columns(((pl.col(pi.indicators.CLOSE_COLUMN) > lower_limit) & (pl.col(pi.indicators.CLOSE_COLUMN) < upper_limit)).alias(close_within_limits))

In [37]:
#per moving_average_comparision, (3, 5), (10,20) and (15,30) are best pairs
sma3 = pi.indicators.simple_moving_average(df, 3)
df = sma3.df
sma5 = pi.indicators.simple_moving_average(df, 5)
df = sma5.df
sma10 = pi.indicators.simple_moving_average(df, 10)
df = sma10.df
sma20 = pi.indicators.simple_moving_average(df, 20)
df = sma20.df
sma15 = pi.indicators.simple_moving_average(df, 15)
df = sma15.df
sma30 = pi.indicators.simple_moving_average(df, 30)
df = sma30.df

In [39]:
sma3_cross_5 = pi.indicators.crossover_up(df, sma3.column, sma5.column)
df = sma3_cross_5.df
sma10_cross_20 = pi.indicators.crossover_up(df, sma10.column, sma20.column)
df = sma10_cross_20.df
sma15_cross_30 = pi.indicators.crossover_up(df, sma15.column, sma30.column)
df = sma15_cross_30.df

In [40]:
russell = df.filter(pl.col(pi.indicators.SYMBOL_COLUMN).is_in(pi.Russell1000_2020))

In [29]:
#df = df.filter(pl.col(pi.indicators.SYMBOL_COLUMN).is_in(pi.SnP500_2020))

In [57]:
buy_cols = []
trades = []

In [58]:
sma3_cross_and_sma10 = "sma3_sma10"
buy_cols.append(sma3_cross_and_sma10)
df = df.with_columns((pl.col(sma3_cross_5.column) & pl.col(sma10_cross_20.column)).alias(sma3_cross_and_sma10))
trades.append(pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(df, sma3_cross_and_sma10, trailing_stop_bars=2)))

In [59]:
sma3_cross_and_sma15 = "sma3_sma15"
buy_cols.append(sma3_cross_and_sma15)
df = df.with_columns((pl.col(sma3_cross_5.column) & pl.col(sma15_cross_30.column)).alias(sma3_cross_and_sma15))
trades.append(pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(df, sma3_cross_and_sma15, trailing_stop_bars=2)))

In [60]:
sma10_cross_and_sma15 = "sma10_sma15"
buy_cols.append(sma10_cross_and_sma15)
df = df.with_columns((pl.col(sma10_cross_20.column) & pl.col(sma15_cross_30.column)).alias(sma10_cross_and_sma15))
trades.append(pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(df, sma10_cross_and_sma15, trailing_stop_bars=2)))

In [61]:
pi.summarize.trades.compare_strategies(trades, buy_cols)

Gain/Loss%Sum,Winners,Losers,Trades,Win_Ratio,Average_Gain/Loss%,Average_Length,highest_win,Highest%,lowest_loss,Lowest%,Identifier
f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,str
485.37,400,646,1050,0.380952,0.462257,4.961905,31.5,58.58,-13.36,-24.19,"""sma3_sma10"""
364.6,237,388,629,0.376789,0.57965,4.914149,34.61,47.5,-11.18,-14.16,"""sma3_sma15"""
-113.16,43,107,150,0.286667,-0.7544,4.626667,8.95,23.4,-14.19,-16.98,"""sma10_sma15"""


In [32]:
russell_results = [pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(russell, buy_indicator.column, trailing_stop_bars=2)) for buy_indicator in buy_indicators]

In [33]:
pi.summarize.trades.compare_strategies(russell_results, [str(x) for x in sma_pairs])

Gain/Loss%Sum,Winners,Losers,Trades,Win_Ratio,Average_Gain/Loss%,Average_Length,highest_win,Highest%,lowest_loss,Lowest%,Identifier
f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,str
14986.99,23597,45167,69057,0.341703,0.217023,4.698524,144.23,520.19,-90.91,-91.01,"""(3, 5)"""
-4247.27,13564,31715,45537,0.297868,-0.093271,4.423436,95.21,127.84,-90.93,-91.03,"""(5, 8)"""
1231.29,5321,11902,17299,0.30759,0.071177,4.538875,79.01,111.95,-47.89,-52.73,"""(8, 21)"""
2800.15,5432,12185,17716,0.306615,0.158058,4.499774,64.42,87.52,-52.96,-62.46,"""(10, 20)"""
2075.17,3632,8029,11717,0.309977,0.177108,4.459674,72.36,143.35,-47.89,-50.95,"""(15, 30)"""
-681.59,859,2525,3403,0.252424,-0.200291,4.06553,36.71,43.07,-47.89,-50.95,"""(50, 100)"""


In [34]:
russell_results[4].df

enter/exit_column,Symbol,Start,End,Entry_Price,Exit_Price,Low,High,Length,Gain/Loss,Gain/Loss%,Highest%,Lowest%
i32,str,datetime[μs],datetime[μs],f64,f64,f64,f64,u32,f64,f64,f64,f64
1,"""A""",2020-04-14 00:00:00,2020-04-16 00:00:00,78.830002,77.0,76.470001,79.199997,3,-1.830002,-2.32,0.47,-2.99
2,"""A""",2020-07-15 00:00:00,2020-07-24 00:00:00,92.879997,95.809998,91.080002,98.089996,8,2.93,3.15,5.61,-1.94
3,"""A""",2020-09-30 00:00:00,2020-10-02 00:00:00,100.940002,100.209999,99.809998,102.410004,3,-0.730003,-0.72,1.46,-1.12
4,"""A""",2021-02-23 00:00:00,2021-02-26 00:00:00,122.419998,121.93,120.410004,125.410004,4,-0.489998,-0.4,2.44,-1.64
5,"""A""",2021-03-30 00:00:00,2021-04-07 00:00:00,124.839996,129.0,124.410004,131.860001,6,4.160004,3.33,5.62,-0.34
6,"""A""",2021-06-04 00:00:00,2021-06-25 00:00:00,137.899994,146.429993,137.369995,148.330002,16,8.529999,6.19,7.56,-0.38
7,"""A""",2021-11-04 00:00:00,2021-11-05 00:00:00,157.649994,154.570007,152.429993,158.929993,2,-3.079987,-1.95,0.81,-3.31
8,"""A""",2021-12-30 00:00:00,2022-01-03 00:00:00,160.880005,159.0,153.929993,162.619995,3,-1.880005,-1.17,1.08,-4.32
9,"""A""",2022-03-23 00:00:00,2022-03-31 00:00:00,134.179993,134.520004,132.289993,139.320007,7,0.340012,0.25,3.83,-1.41


In [35]:
russell.filter(pl.col(pi.indicators.DATE_COLUMN).is_between(datetime(2020,4,22),datetime(2020,4,27)) & (pl.col(pi.indicators.SYMBOL_COLUMN) == "A"))

Date,Open,High,Low,Close,Adj Close,Volume,Symbol,close_over3_under250,SMA3_Close,SMA5_Close,SMA8_Close,SMA21_Close,SMA10_Close,SMA20_Close,SMA15_Close,SMA30_Close,SMA50_Close,SMA100_Close,SMA3_Close_cross_up_SMA5_Close,SMA5_Close_cross_up_SMA8_Close,SMA8_Close_cross_up_SMA21_Close,SMA10_Close_cross_up_SMA20_Close,SMA15_Close_cross_up_SMA30_Close,SMA50_Close_cross_up_SMA100_Close
datetime[μs],f64,f64,f64,f64,f64,i64,str,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool,bool,bool
2020-04-22 00:00:00,74.75,75.580002,73.779999,75.169998,73.746544,2347400,"""A""",true,75.43,76.918001,77.085001,74.125715,77.170001,74.482001,75.448001,71.904334,75.5774,null,false,false,false,false,false,null
2020-04-23 00:00:00,76.599998,77.489998,75.050003,75.089996,73.668068,2097700,"""A""",true,74.669998,76.186,76.945001,74.510953,77.010001,74.786501,75.859334,72.112667,75.3774,null,false,false,false,false,false,null
2020-04-24 00:00:00,75.620003,75.910004,74.019997,75.75,74.315559,1654300,"""A""",true,75.336665,75.425999,76.56,74.832381,76.752,74.888,76.09,72.523667,75.1838,null,false,false,false,false,false,null
2020-04-27 00:00:00,76.349998,76.620003,75.839996,76.25,74.806091,1295800,"""A""",true,75.696665,75.201999,76.46,74.952858,76.756001,75.155,76.478667,72.758667,74.9994,null,true,false,false,false,false,null
